# Практическая работа №4: Элементы корреляционного анализа. Проверка статистической гипотезы о равенстве коэффициента корреляции нулю

Выполнила студентка гр. 1384 Шиняева Анастасия. Вариант №4

## Цель работы

Освоение основных понятий, связанных с корреляционной зависимостью между случайными величинами, статистическими гипотезами и проверкой их «справедливости».

## Основные теоретические положения

**Коэффициент корреляции:**

$$ r = \frac{\text{cov}(X, Y)}{\sigma_X \cdot \sigma_Y}, $$

где $ \text{cov}(X, Y) $ — ковариация, $ \sigma_X $ и $ \sigma_Y $ — стандартные отклонения $ X $ и $ Y $.

Для коэффициента корреляции справедливо соотношение $ |r_{xy}| < 1 $

**Коэффициент корреляции через условные варианты:**

$$ \overline{r}_{xy} = \frac{\sum_{i=1}^{K_y} \sum_{j=1}^{K_x} n_{ij} y_i x_j - N \overline{x}_G \overline{y}_G}{N S_x S_y} $$

**Преобразование Фишера (z-преобразование):**

$$ z = \frac{1}{2} \ln\left(\frac{1 + \overline{r}_{xy}}{1 - \overline{r}_{xy}}\right) $$

**Стандартная ошибка для z-значения:**

$$ \overline{\sigma}_{z} = \frac{1}{\sqrt{N - 3}} $$

**Доверительный интервал для z-значения:**

$$ z_{\text{lower}} = z - z_{\gamma} \cdot \overline{\sigma}_{z} $$
$$ z_{\text{upper}} = z + z_{\gamma} \cdot \overline{\sigma}_{z} $$

**Обратное преобразование Фишера:**

$$ r_{\text{lower}} = \frac{e^{2z_{\text{lower}}} - 1}{e^{2z_{\text{lower}}} + 1} $$
$$ r_{\text{upper}} = \frac{e^{2z_{\text{upper}}} - 1}{e^{2z_{\text{upper}}} + 1} $$

**T-статистика для проверки гипотезы о равенстве коэффициента корреляции нулю:**

$$ T = \frac{\overline{r}_{xy} \sqrt{N - 2}}{\sqrt{1 - \overline{r}_{xy}^2}} $$

## Постановка задачи

Из заданной генеральной совокупности сформировать выборку по второму признаку. Провести статистическую обработку второй выборки в объёме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса и коэффициента вариации). Для системы двух случайных величин X (первый признак) и Y (второй признак) сформировать двумерную выборку и найти статистическую оценку коэффициента корреляции, построить доверительный интервал для коэффициента корреляции и осуществить проверку статистической гипотезы о равенстве коэффициента корреляции нулю. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы

Перенесем необходимые данные из первых практических работ для выполнения текущей.

In [540]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('/data.csv', header=None, names=['nu', 'E']) # Выборка

ranked_data = data['E'].sort_values() # Ранжированный ряд

min_value_E = data['E'].min() # Минимальное значение
max_value_E = data['E'].max() # Максимальное значение

bin_edges = 7 # Количество интервалов
h = (max_value_E - min_value_E) / bin_edges # Ширина интервала

intervals_E = [min_value_E + i * h for i in range(bin_edges + 1)] # Границы интервала

absolute_freq, bin_edges = np.histogram(data['E'], bins=bin_edges) # Абсолютная частота
relative_freq = absolute_freq / len(data) # Относительная частота

bin_mid_E = (bin_edges[:-1] + bin_edges[1:]) / 2 # Середины интервалов

cumulative_absolute_freq = np.cumsum(absolute_freq) # Накопленная абсолютная частота
cumulative_relative_freq = np.cumsum(relative_freq) # Накопленная относительная частота

C = bin_mid_E[len(bin_mid_E) // 2]
u = (bin_mid_E - C) / h # Условные варианты

n_u1 = relative_freq * u**1 # Условные моменты
n_u2 = relative_freq * u**2
n_u3 = relative_freq * u**3
n_u4 = relative_freq * u**4

n_u1_4 = relative_freq * (u+1)**4 # Сумма условных моментов

M_1 = sum(n_u1)  # Условные эмпирические моменты
M_2 = sum(n_u2)
M_3 = sum(n_u3)
M_4 = sum(n_u4)

m_1 = 0 # Центральные эмпирические моменты
m_2 = (M_2 - M_1**2)*h**2
m_3 = (M_3 - 3*M_1*M_2 + 2*M_1**3)*h**3
m_4 = (M_4 - 4*M_1*M_3 + 6*M_1**2*M_2 - 3*M_1**4)*h**4

x_avg_b = M_1*h + C # Выборочное среднее
D_b = m_2 # Выборочная дисперсия

sigma_b = np.sqrt(D_b) # Выборочное СКО

s_2 = len(data['E']) / (len(data['E']) - 1) * D_b # Исправленная выборочная дисперсия
s = np.sqrt(s_2) # Исправленное СКО

A_s = m_3 / s**3 # Статистическая оценка коэффициента асимметрии
E = m_4 / s**4 - 3 # Статистическая оценка коэффициента эксцесса

n = len(data['E'])
i = np.where(cumulative_absolute_freq >= n / 2)[0][0]
L = intervals_E[i]
F = cumulative_absolute_freq[i - 1] if i > 0 else 0
f = absolute_freq[i]

M_e = L + h * ((n / 2 - F) / f) # Медиана
M_o = 3 * M_e - 2 * x_avg_b # Мода
V = (sigma_b / x_avg_b) * 100 # Коэффициент вариации

In [541]:
results_table = pd.DataFrame({
    'Параметр': ['Математическое ожидание', 'Дисперсия','СКО', 'Асимметрия',
                 'Эксцесс', 'Медиана', 'Мода', 'Коэффициент вариации'],
    'Значение': [x_avg_b, s_2, s, A_s, E, M_e, M_o, V]
}, index=range(1, 9))

# print(results_table)

In [542]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('/data.csv', header=None, names=['nu', 'E']) # Выборка

ranked_data = data['nu'].sort_values() # Ранжированный ряд

min_value_nu = data['nu'].min() # Минимальное значение
max_value_nu = data['nu'].max() # Максимальное значение

bin_edges = 7 # Количество интервалов
h_nu = (max_value_nu - min_value_nu) / bin_edges # Ширина интервала

intervals_nu = [min_value_nu + i * h_nu for i in range(bin_edges + 1)] # Границы интервала

absolute_freq, bin_edges = np.histogram(data['nu'], bins=bin_edges) # Абсолютная частота
relative_freq = absolute_freq / len(data) # Относительная частота

bin_mid_nu = (bin_edges[:-1] + bin_edges[1:]) / 2 # Середины интервалов

cumulative_absolute_freq = np.cumsum(absolute_freq) # Накопленная абсолютная частота
cumulative_relative_freq = np.cumsum(relative_freq) # Накопленная относительная частота

C = bin_mid_nu[len(bin_mid_nu) // 2]
u = (bin_mid_nu - C) / h_nu # Условные варианты

n_u1 = relative_freq * u**1 # Условные моменты
n_u2 = relative_freq * u**2
n_u3 = relative_freq * u**3
n_u4 = relative_freq * u**4

n_u1_4 = relative_freq * (u+1)**4 # Сумма условных моментов

M_1 = sum(n_u1)  # Условные эмпирические моменты
M_2 = sum(n_u2)
M_3 = sum(n_u3)
M_4 = sum(n_u4)

m_1 = 0 # Центральные эмпирические моменты
m_2 = (M_2 - M_1**2)*h_nu**2
m_3 = (M_3 - 3*M_1*M_2 + 2*M_1**3)*h_nu**3
m_4 = (M_4 - 4*M_1*M_3 + 6*M_1**2*M_2 - 3*M_1**4)*h_nu**4

x_avg_b = M_1*h_nu + C # Выборочное среднее
D_b = m_2 # Выборочная дисперсия

sigma_b = np.sqrt(D_b) # Выборочное СКО

s_2 = len(data['nu']) / (len(data['nu']) - 1) * D_b # Исправленная выборочная дисперсия
s = np.sqrt(s_2) # Исправленное СКО

A_s = m_3 / s**3 # Статистическая оценка коэффициента асимметрии
E = m_4 / s**4 - 3 # Статистическая оценка коэффициента эксцесса

n = len(data['nu'])
i = np.where(cumulative_absolute_freq >= n / 2)[0][0]
L = intervals_nu[i]
F = cumulative_absolute_freq[i - 1] if i > 0 else 0
f = absolute_freq[i]

M_e = L + h_nu * ((n / 2 - F) / f) # Медиана
M_o = 3 * M_e - 2 * x_avg_b # Мода
V = (sigma_b / x_avg_b) * 100 # Коэффициент вариации

In [543]:
results_table = pd.DataFrame({
    'Параметр': ['Математическое ожидание', 'Дисперсия','СКО', 'Асимметрия',
                 'Эксцесс', 'Медиана', 'Мода', 'Коэффициент вариации'],
    'Значение': [x_avg_b, s_2, s, A_s, E, M_e, M_o, V]
}, index=range(1, 9))

# print(results_table)

Оформим вычисления в таблицу:

Таблица 1

|        	| Математическое ожидание 	| Дисперсия 	|   СКО   	| Асимметрия 	| Эксцесс 	|  Медиана 	|   Мода   	| Коэффициент вариации 	|
|:-------:|:------------------------:|:----------:|:--------:|:-----------:|:--------:|:---------:|:---------:|:---------------------:|
| X (nu) 	|         445.3020        	| 3356.4403 	| 57.9348 	|   -0.0318  	| -0.5621 	| 447.1400 	| 450.8160 	|        12.9457       	|
|  Y (E) 	|         127.1668        	|  605.7634 	| 24.6123 	|   -0.2296  	| -0.3412 	| 128.2154 	| 130.3127 	|        19.2583       	|

Выводы: см. практическая работа №2.

Построим двумерный интервальный вариационный ряд.

In [544]:
intervals_nu[-1] += 0.001
intervals_E[-1] += 0.001

df_int['X'] = pd.cut(df_int['nu'], bins=intervals_nu, right=False, labels=bin_mid_nu)
df_int['Y'] = pd.cut(df_int['E'], bins=intervals_E, right=False, labels=bin_mid_E)

two_table = df_int.groupby(['Y', 'X'], observed=False).size().unstack(fill_value=0)

# print(two_table)
# print("\nСумма таблицы:", two_table.sum().sum())

Оформим вычисления в таблицу:

Таблица 2

|  $ Y $  	|       	|       	|       	| $ X $ 	|       	|       	|       	|         	|
|:--------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:--------:|
|         	| 339.5 	| 378.5 	| 417.5 	| 456.5 	| 495.5 	| 534.5 	| 573.5 	| $ n_y $ 	|
|  73.28  	|   4   	|   1   	|   0   	|   0   	|   0   	|   0   	|   0   	|    5    	|
|  90.84  	|   4   	|   5   	|   0   	|   0   	|   0   	|   0   	|   0   	|    9    	|
|  108.39 	|   1   	|   5   	|   9   	|   2   	|   0   	|   0   	|   0   	|    17   	|
|  125.95 	|   0   	|   1   	|   13  	|   14  	|   3   	|   0   	|   0   	|    31   	|
|  143.51 	|   0   	|   0   	|   1   	|   9   	|   11  	|   3   	|   0   	|    24   	|
|  161.06 	|   0   	|   0   	|   0   	|   0   	|   7   	|   4   	|   1   	|    12   	|
|  178.62 	|   0   	|   0   	|   0   	|   0   	|   1   	|   0   	|   2   	|    3    	|
| $ n_x $ 	|   5   	|   12  	|   23  	|   25  	|   22  	|   7   	|   3   	|   101   	|

По полученному двумерному интервальному вариационному ряду построим корреляционную таблицу.

In [545]:
corr_table = two_table.to_numpy()

X_k = len(bin_mid_nu)
Y_k = len(bin_mid_E)

X_i = [[0] * X_k for _ in range(Y_k)]
Y_j = [[0] * Y_k for _ in range(X_k)]

for i in range(Y_k):
    for j in range(X_k):
        X_i[i][j] = bin_mid_nu[j] * table[i][j]
        Y_j[i][j] = bin_mid_E[i] * table[i][j]

Y_j_sum = [sum([row[j] for row in Y_j]) for j in range(X_k)]
X_i_sum = [sum(row) for row in X_i]

yiXi = [bin_mid_E[i] * X_i_sum[i] for i in range(Y_k)]
xjYj = [bin_mid_nu[j] * Y_j_sum[j] for j in range(X_k)]

# print("Y_j:", Y_j_sum)
# print("X_i:", X_i_sum)
# print("yiXi:", yiXi, sum(yiXi))
# print("xjYj:", xjYj, sum(xjYj))
# print("\n")

# def rnd(value, decimals=2):
#     return round(value, decimals)

# for i in range(Y_k):
#     print(rnd(bin_mid_E[i]), end=' | ')
#     for j in range(X_k):
#         print(f"{rnd(Y_j[i][j], 2)} \\ {rnd(X_i[i][j], 2)}", end=' | ')
#     print()

Оформим вычисления в таблицу:

Таблица 3

|    $ Y $    	|                 	|                 	|                 	|       $ X $      	|                  	|                  	|                	|              	|              	|
|:------------:|:----------------:|:----------------:|:----------------:|:-----------------:|:-----------------:|:-----------------:|:---------------:|:-------------:|:-------------:|
|             	|      339.5      	|      378.5      	|      417.5      	|       456.5      	|       495.5      	|       534.5      	|      573.5     	|    $ X_i $   	|  $ y_i X_i $ 	|
|    73.28    	| 293.11 \ 1358.0 	| 293.11 \ 1514.0 	|  73.28 \ 417.5  	|     0.0 \ 0.0    	|     0.0 \ 0.0    	|     0.0 \ 0.0    	|    0.0 \ 0.0   	|    3289.5    	|  241049.8607 	|
|    90.84    	|  90.84 \ 339.5  	| 454.18 \ 1892.5 	| 454.18 \ 2087.5 	|   90.84 \ 456.5  	|     0.0 \ 0.0    	|     0.0 \ 0.0    	|    0.0 \ 0.0   	|    4776.0    	|  433831.3714 	|
|    108.39   	|    0.0 \ 0.0    	|    0.0 \ 0.0    	| 975.54 \ 3757.5 	| 1409.11 \ 5934.5 	|  108.39 \ 495.5  	|     0.0 \ 0.0    	|    0.0 \ 0.0   	|    10187.5   	| 1104252.2321 	|
|    125.95   	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|  251.9 \ 835.0  	|  1763.3 \ 6391.0 	| 1133.55 \ 4459.5 	|     0.0 \ 0.0    	|    0.0 \ 0.0   	|    11685.5   	|  1471788.725 	|
|    143.51   	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|  430.52 \ 1369.5 	| 1578.58 \ 5450.5 	| 1004.55 \ 3741.5 	| 143.51 \ 573.5 	|    11135.0   	| 1597952.0357 	|
|    161.06   	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|     0.0 \ 0.0    	|  483.19 \ 1486.5 	|  644.26 \ 2138.0 	|    0.0 \ 0.0   	|    3624.5    	|  583777.5036 	|
|    178.62   	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|    0.0 \ 0.0    	|     0.0 \ 0.0    	|     0.0 \ 0.0    	|  178.62 \ 534.5  	| 178.62 \ 573.5 	|    1108.0    	|  197912.5429 	|
|   $ Y_j $   	|      383.95     	|     747.2929    	|    1754.8929    	|     3693.7643    	|     3303.7143    	|     1827.4286    	|    322.1286    	|       -      	| 5630564.2714 	|
| $ x_j Y_j $ 	|    130351.025   	|   282850.3464   	|   732667.7679   	|   1686203.3964   	|   1636990.4286   	|    976760.5714   	|   184740.7357  	| 5630564.2714 	|       -      	|

Выводы: совпадение сумм $ x_j Y_j $ и $ y_i X_i $подтверждает, что расчеты выполнены правильно, и данные обработаны корректно.

Исходя из результатов корреляционной таблицы вычислим значение выборочного коэффициента корреляции двумя способами: с помощью стандартной формулы и с помощью условных вариант и убедимся, что результаты совпадают.

In [546]:
X_mid = bin_mid_nu
Y_mid = bin_mid_E
N = np.sum(corr_table)

X_mean = np.sum(X_mid * np.sum(corr_table, axis=0)) / N
Y_mean = np.sum(Y_mid * np.sum(corr_table, axis=1)) / N

nij_yi_xj_sum = 0
for i in range(Y_k):
    for j in range(X_k):
        nij_yi_xj_sum += corr_table[i][j] * Y_mid[i] * X_mid[j]

S_x = np.sqrt(np.sum([(x - X_mean)**2 * np.sum(corr_table[:, j]) for j, x in enumerate(X_mid)]) / N)
S_y = np.sqrt(np.sum([(y - Y_mean)**2 * np.sum(corr_table[i, :]) for i, y in enumerate(Y_mid)]) / N)

cov_XY = 0
for i in range(Y_k):
    for j in range(X_k):
        cov_XY += (X_mid[j] - X_mean) * (Y_mid[i] - Y_mean) * corr_table[i][j]
cov_XY /= N

r_standard = cov_XY / (S_x * S_y)

print("Коэффициент корреляции с помощью стандартной формулы:", r_standard)

nij_yi_xj_sum = 0
for i in range(Y_k):
    for j in range(X_k):
        nij_yi_xj_sum += corr_table[i][j] * Y_mid[i] * X_mid[j]

r_conditional = (nij_yi_xj_sum - N * X_mean * Y_mean) / (N * S_x * S_y)

print("Коэффициент корреляции с помощью стандартной формулы:", r_conditional)

Коэффициент корреляции с помощью стандартной формулы: 0.8788242550092751
Коэффициент корреляции с помощью стандартной формулы: 0.8788242550092743


Выводы: результаты совпали. Значение 0.8788 близко к 1, что говорит о сильной положительной линейной связи между величинами X и Y. Коэффициент корреляции отличен от нуля => величины X и Y коррелированы => зависимы.

Построим доверительный интервал для коэффициента корреляции при уровне значимости $ γ∈{0.95,0.99} $.

In [547]:
from scipy.stats import norm

gamma_values = [0.95, 0.99]

z = 0.5 * np.log((1 + r_standard) / (1 - r_standard))

S_z = 1 / np.sqrt(N - 3)

def fisher_inverse(z):
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)

for gamma in gamma_values:
    z_gamma = norm.ppf((1 + gamma) / 2)

    z_lower = z - z_gamma * S_z
    z_upper = z + z_gamma * S_z

    r_lower = fisher_inverse(z_lower)
    r_upper = fisher_inverse(z_upper)

    print(f"Доверительный интервал для коэффициента корреляции при γ = {gamma}: ({r_lower:.4f}, {r_upper:.4f})")

Доверительный интервал для коэффициента корреляции при γ = 0.95: (0.8251, 0.9168)
Доверительный интервал для коэффициента корреляции при γ = 0.99: (0.8042, 0.9262)


Выводы: нижние границы интервалов (0.8251 и 0.8042) значительно выше нуля, а верхние границы интервалов (0.9168 и 0.9262) близки к 1, что указывают на сильную положительную линейную связь между переменными X и Y.

Связь между переменными статистически значима, так как доверительные интервалы не включают ноль.

Осуществим проверку статистической гипотезы о равенстве коэффициента корреляции нулю при заданном уровне значимости $ α=0.05 $.

In [548]:
from scipy.stats import t

alpha = 0.05

T_observed = r_standard * np.sqrt(N - 2) / np.sqrt(1 - r_standard**2)

df = N - 2
t_critical = t.ppf(1 - alpha / 2, df)

print("Наблюдаеиое значение T:", T_observed)
print("Критическое значение t:", t_critical)

Наблюдаеиое значение T: 18.326043102092896
Критическое значение t: 1.9842169515086827


Выводы: $ T_{набл} > t_{крит} $ нулевая гипотеза $ H_0 $ отвергается - коэффициент корреляции значимо отличается от нуля.

## Выводы

В ходе выполнения практической работы были изучены основные понятия, связанные с корреляционной зависимостью между случайными величинами, статистическими гипотезами и проверкой их «справедливости».

Результаты показали, что между исследуемыми величинами X и Y существует сильная положительная линейная связь, что подтверждается значением коэффициента корреляции r=0.87. Коэффициент корреляции отличен от нуля, а значит, величины X и Y коррелированы и зависимы. Полученные доверительные интервалы для коэффициента корреляции при уровнях значимости γ=0.95 и γ=0.99 указывают на высокую степень уверенности в наличии сильной связи между велечинами.

Проверка статистической гипотезы о равенстве коэффициента корреляции нулю на уровне значимости α=0.05 показала, что нулевая гипотеза отвергается. Это означает, что коэффициент корреляции значимо отличается от нуля, и связь между величинами X и Y является статистически значимой.